<a href="https://colab.research.google.com/github/Sagnik-Nandi/PDFQueryBot---Chatbot-over-PDFs-using-RAG/blob/main/final%20project/llama_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.6 MB/s eta 0:00:00


In [10]:
!pip install langchain -q
!pip install langchain_chroma -q
!pip install langchain_community -q
!pip install langchain_groq -q
!pip install grandalf -q
!pip install numpy -q
!pip install pandas -q
!pip install pypdf -q
!pip install sentence-transformers -q #takes 2 min to exec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 4.7 MB/s eta 0:00:00


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
import os
os.environ["GROQ_API_KEY"] = "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"
os.environ["TOKENIZERS_PARALLELISM"] = "false" # To suppress huggingface warnings

import warnings
warnings.filterwarnings("ignore")

from groq import Groq

In [8]:
from langchain_groq import ChatGroq
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings


embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
rag_llm = ChatGroq(model="llama3-8b-8192") # Used for RAG

<ipython-input-8-ce53ba393791>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#Loading
pdf_path = "./ugrulebook.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load() # list of pages

# Splitting
# split a long document into smaller chunks that can fit into your model's context window
# 2 hyperparameters : chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n",
        "\n",
        " ",
        "",],
    chunk_size=2000,
    chunk_overlap=200
  )
docs_spl = text_splitter.split_documents(docs)
len(docs_spl)


92

In [19]:

from langchain_chroma import Chroma

# Storing
vectorstore = Chroma.from_documents(docs_spl, embedding=embed_model, collection_name="groq_rag") # takes one min to run
retriever = vectorstore.as_retriever()
print(f"Documents indexed: {len(docs_spl)}")

Documents indexed: 92


In [20]:
await retriever.ainvoke("What are the eligibility criteria for applying for a change of branch/ programme?")

# splitting can be improved ... eg \n is part of a word

[Document(id='a92d50a2-2380-4870-b938-6311af125163', metadata={'author': 'pritap', 'creationdate': '2025-01-27T11:14:55+05:30', 'creator': 'Microsoft® Word 2016', 'moddate': '2025-01-27T11:14:55+05:30', 'page': 34, 'page_label': '35', 'producer': 'Microsoft® Word 2016', 'source': './ugrulebook.pdf', 'total_pages': 52}, page_content='there are valid requests. \nE) All changes of branch can b e effected only once at the beginning of the second academic \nyear. No application for change of branch during the subsequent academic years will be \nentertained. \nF) Branch change decisions will be final and will not be reversed. \nG) To run the LASE programme, the minimum student strength for the LASE programme should \nbe 10. If less than 10 students are allotted the LASE programme after branch change then \nthe result will be considered as null and void.'),
 Document(id='2384459f-b0f3-4b6f-9835-45ef5b2289f6', metadata={'author': 'pritap', 'creationdate': '2025-01-27T11:14:55+05:30', 'creator'

In [21]:
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from typing import List, Dict

RAG_SYSTEM_PROMPT = """\
You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context given within delimiters to answer the human's questions.
```
{context}
```
If you don't know the answer, just say that you don't know.\
""" # adapted from https://smith.langchain.com/hub/rlm/rag-prompt-llama3

RAG_HUMAN_PROMPT = "{input}"

RAG_PROMPT = ChatPromptTemplate.from_messages([
    ("system", RAG_SYSTEM_PROMPT),
    ("human", RAG_HUMAN_PROMPT)
])

def format_docs(docs: List[Document]):
    """Format the retrieved documents"""
    return "\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs, # Use retriever to retrieve docs from vectorstore -> format the documents into a string
        "input": RunnablePassthrough() # Propogate the 'input' variable to the next step
    }
    | RAG_PROMPT # format prompt with 'context' and 'input' variables
    | rag_llm # get response from LLM using the formatteed prompt
    | StrOutputParser() # Parse through LLM response to get only the string response

)

In [24]:
await rag_chain.ainvoke("How many candidates does IIT Bombay take in annually?")

'According to the provided context, IIT Bombay on an average annually admits:\n\n* More than 1000 candidates for undergraduate programmes (B.Tech./ Dual Degree and B.S.) through the Joint Entrance Examination (JEE)\n* More than 30 candidates for B.Des. Programme through the Undergraduate Common Entrance Exam for Design (UCEED)\n* Around 300 candidates for the M.Sc. and the M.Sc. Ph.D. Dual Degree programmes\n* More than 1000 candidates for postgraduate programmes\n* Around 300 candidates for Ph.D. programmes'